# Price Performance Service - Stock vs Sector vs Market

Compare stock performance against sector and market benchmarks across multiple time periods.

## Overview

The **Price Performance Service** enables relative performance analysis:

- **Three-Way Comparison**: Stock vs Sector vs Market (SET)
- **Time Periods**: 5-day, 1-month, 3-month, 6-month, YTD
- **Valuation Metrics**: P/E, P/B, turnover ratio for each entity
- **Relative Performance**: Calculate outperformance/underperformance

### When to Use

- Sector rotation strategies
- Relative strength analysis
- Performance attribution
- Alpha calculation (stock vs benchmark)
- Market timing decisions

In [ ]:
!pip install settfex

In [ ]:
import asyncio
from settfex.services.set import get_price_performance
import pandas as pd

print("Libraries imported!")

## Basic Usage

In [ ]:
# Fetch price performance
perf = await get_price_performance("MINT")

print(f"Price Performance Analysis for {perf.stock.symbol}\n")

print("Stock Performance:")
print(f"  5-Day: {perf.stock.five_day_percent_change:+.2f}%")
print(f"  1-Month: {perf.stock.one_month_percent_change:+.2f}%")
print(f"  3-Month: {perf.stock.three_month_percent_change:+.2f}%")
print(f"  6-Month: {perf.stock.six_month_percent_change:+.2f}%")
print(f"  YTD: {perf.stock.ytd_percent_change:+.2f}%")
print(f"  P/E: {perf.stock.pe_ratio}, P/B: {perf.stock.pb_ratio}")

print(f"\nSector ({perf.sector.symbol}) Performance:")
print(f"  YTD: {perf.sector.ytd_percent_change:+.2f}%")

print(f"\nMarket ({perf.market.symbol}) Performance:")
print(f"  YTD: {perf.market.ytd_percent_change:+.2f}%")

# Calculate relative performance
vs_sector = perf.stock.ytd_percent_change - perf.sector.ytd_percent_change
vs_market = perf.stock.ytd_percent_change - perf.market.ytd_percent_change

print(f"\nRelative Performance (YTD):")
print(f"  vs Sector: {vs_sector:+.2f}%")
print(f"  vs Market: {vs_market:+.2f}%")

## Advanced Usage - Multi-Stock Comparison

In [ ]:
async def relative_performance_analysis(symbols: list[str]):
    """
    Analyze relative performance for multiple stocks.
    """
    tasks = [get_price_performance(symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    analysis = []
    for perf in results:
        if isinstance(perf, Exception):
            continue
        
        vs_sector = perf.stock.ytd_percent_change - perf.sector.ytd_percent_change
        vs_market = perf.stock.ytd_percent_change - perf.market.ytd_percent_change
        
        analysis.append({
            "symbol": perf.stock.symbol,
            "stock_ytd": perf.stock.ytd_percent_change,
            "sector_ytd": perf.sector.ytd_percent_change,
            "market_ytd": perf.market.ytd_percent_change,
            "vs_sector": vs_sector,
            "vs_market": vs_market,
            "pe_ratio": perf.stock.pe_ratio
        })
    
    df = pd.DataFrame(analysis).sort_values('vs_market', ascending=False)
    return df

# Analyze blue chips
symbols = ["PTT", "KBANK", "CPALL", "AOT", "BBL", "SCB", "MINT"]
rel_perf = await relative_performance_analysis(symbols)

print("Relative Performance Analysis (YTD):\n")
print(rel_perf.to_string(index=False))

## Use Case: Sector Leaders Identification

In [ ]:
# Find stocks outperforming their sector
sector_leaders = rel_perf[rel_perf['vs_sector'] > 0].copy()
sector_leaders = sector_leaders.sort_values('vs_sector', ascending=False)

print(f"Sector Leaders (Outperforming Sector): {len(sector_leaders)} stocks\n")
print(sector_leaders[['symbol', 'stock_ytd', 'sector_ytd', 'vs_sector']].to_string(index=False))

## Use Case: Alpha Calculation

In [ ]:
# Calculate alpha (excess return over market)
print("Alpha Analysis (Stock Return - Market Return):\n")

alpha_df = rel_perf[['symbol', 'stock_ytd', 'market_ytd', 'vs_market']].copy()
alpha_df.columns = ['Symbol', 'Stock YTD', 'Market YTD', 'Alpha']
alpha_df = alpha_df.sort_values('Alpha', ascending=False)

print(alpha_df.to_string(index=False))

positive_alpha = len(alpha_df[alpha_df['Alpha'] > 0])
print(f"\nStocks with Positive Alpha: {positive_alpha}/{len(alpha_df)}")

## Use Case: Multi-Period Momentum

In [ ]:
# Analyze momentum across periods
perf = await get_price_performance("CPALL")

momentum_data = {
    "5-Day": perf.stock.five_day_percent_change,
    "1-Month": perf.stock.one_month_percent_change,
    "3-Month": perf.stock.three_month_percent_change,
    "6-Month": perf.stock.six_month_percent_change,
    "YTD": perf.stock.ytd_percent_change
}

print(f"Momentum Analysis for {perf.stock.symbol}:\n")
for period, change in momentum_data.items():
    print(f"{period:10} {change:+7.2f}%")

# Check if trending
all_positive = all(v > 0 for v in momentum_data.values() if v is not None)
print(f"\nConsistent Uptrend: {'Yes' if all_positive else 'No'}")

## Next Steps

- **[Trading Statistics](09_trading_statistics.ipynb)** - Detailed period statistics
- **[Highlight Data](02_highlight_data.ipynb)** - Current metrics

**Documentation**: `/docs/settfex/services/set/price_performance.md`